<a href="https://colab.research.google.com/github/nour-ezzehi/Quiz-generator/blob/main/Quiz_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import spacy
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random

In [23]:
nlp = spacy.load('en_core_web_sm')

model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def process_text(text):
    doc = nlp(text)

    entities = [(ent.text, ent.label_) for ent in doc.ents]

    sentences = [sent.text for sent in doc.sents]

    return sentences, entities

In [24]:
def generate_question(text):
    input_text = f"generate a question: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output = model.generate(inputs['input_ids'], max_length=50)
    question = tokenizer.decode(output[0], skip_special_tokens=True)
    return question

In [25]:
def generate_mcq(question, correct_answer):
    distractors = [correct_answer, "incorrect_answer_1", "incorrect_answer_2", "incorrect_answer_3"]
    random.shuffle(distractors)

    return {
        "question": question,
        "options": distractors,
        "answer": correct_answer
    }
text = "Albert Einstein was born in Ulm, Germany, in 1879. He was a theoretical physicist and is best known for developing the theory of relativity."

sentences, entities = process_text(text)

quiz = []

In [26]:
for sentence in sentences:
    question = generate_question(sentence)
    if len(entities) > 0:
        correct_answer = entities[0][0]
        mcq = generate_mcq(question, correct_answer)
        quiz.append(mcq)

print(quiz)

[{'question': 'a question', 'options': ['Albert Einstein', 'incorrect_answer_1', 'incorrect_answer_2', 'incorrect_answer_3'], 'answer': 'Albert Einstein'}, {'question': 'a question', 'options': ['incorrect_answer_3', 'incorrect_answer_2', 'Albert Einstein', 'incorrect_answer_1'], 'answer': 'Albert Einstein'}]
